In [ ]:
%config InlineBackend.figure_format='retina'
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

rc('figure',figsize=(16,8))
rc('font',size=12)

from scipy.signal import medfilt
from scipy.interpolate import interp1d

from qubicpack.qubicfp import qubicfp
from qubic import fibtools as ft

from importlib import reload
import healpy as hp

import time_domain_tools as tdt


# Trying with QUBIC data from Salta

We use a Synthesized Beam scanning performed on April 16th 2022 in Salta (CalSrc at 140 GHz): 
`2022-04-16_12.37.59__ScanMap_Speed_VE14_FastNoMod`

Or alternatively, a scan a 170 GHz on April 14th:
`2022-04-14_13.17.33__ScanMap_Speed_VE14_FastNoMod`

In [ ]:
mydatadir = '/Users/hamilton/Qubic/Calib-TD/'
thedate = '2022-04-16'
thedata = '2022-04-16_12.37.59__ScanMap_Speed_VE14_FastNoMod'
FreqSrc = 140.

# mydatadir = '/Users/hamilton/Qubic/Calib-TD/'
# thedate = '2022-04-14'
# thedata = '2022-04-14_13.17.33__ScanMap_Speed_VE14_FastNoMod'
# FreqSrc = 170.


filename = mydatadir + '/' + thedate + '/' + thedata

### Read data
a = qubicfp()
a.read_qubicstudio_dataset(filename)

tt, tod = a.tod()

az = a.azimuth()
el = a.elevation()
thk = a.timeaxis(datatype='hk')

TESnum = 33
tod = tod[TESnum-1,:]
del(a)

### We remove tt[0]
tinit = tt[0]
tt -= tinit
thk -= tinit


### First just a glimpse at the data

In [ ]:
rc('figure',figsize=(16,8))
rc('font',size=12)


subplot(2,2,1)
plot(tt, tod)
xlabel('t')
ylabel('TOD')

subplot(2,2,2)
plot(az, el)
xlabel('az')
ylabel('el')

subplot(2,2,3)
plot(thk, az)
xlabel('t')
ylabel('az')

subplot(2,2,4)
plot(thk, el)
xlabel('t')
ylabel('el')


### Identifying scans
For the mapmaking and for many purposes, it will be very useeful to identify each scan:
- a numbering for each back & forth scan
- a region to remove at the end of each scan (bad data due to FLL reset, slowingg down of the moiunt, possibly HWP rotation
- is the scan back or forth ?

The function `identify_scans()` from `time_domain_tools.py` is intended as a first version of this.

In [ ]:
rc('figure',figsize=(20,12))
rc('font',size=12)
reload(tdt)

### Identify scan types and numbers
scantype_hk, azt, elt, scantype = tdt.identify_scans(thk, az, el, tt=tt, doplot=True, plotrange=[0,2000], thr_speedmin=0.1)

In [ ]:
def healpix_map(azt, elt, tod, flags=None, flaglimit=0, nside=128, countcut=0, unseen_val=hp.UNSEEN):
    if flags is None:
        flags = np.zeros(len(azt))
    
    ok = flags <= flaglimit 
    return healpix_map_(azt[ok], elt[ok], tod[ok], nside=nside, countcut=countcut, unseen_val=unseen_val)


def healpix_map_(azt, elt, tod, nside=128, countcut=0, unseen_val=hp.UNSEEN):
    ips = hp.ang2pix(nside, azt, elt, lonlat=True)
    mymap = np.zeros(12*nside**2)
    mapcount = np.zeros(12*nside**2)
    for i in range(len(azt)):
        mymap[ips[i]] += tod[i]
        mapcount[ips[i]] += 1
    unseen = mapcount <= countcut
    mymap[unseen] = unseen_val
    mapcount[unseen] = unseen_val
    mymap[~unseen] = mymap[~unseen] / mapcount[~unseen]
    return mymap, mapcount

In [ ]:
def get_mode(y, nbinsmin=51):
    mm, ss = ft.meancut(y, 4)
    hh = np.histogram(y, bins=int(np.min([len(y) / 30, nbinsmin])), range=[mm - 5 * ss, mm + 5 * ss])
    idmax = np.argmax(hh[0])
    mymode = 0.5 * (hh[1][idmax + 1] + hh[1][idmax])
    return mymode


In [ ]:
def display_one(mapsb, anatype='', sub=(1,1,1), nlo=3, nhi=3, reso=12):
    unseen = (mapsb == hp.UNSEEN)
    mm, ss = ft.meancut(mapsb[~unseen], 3)
    hp.gnomview(mapsb, rot=[0,50], reso=reso, sub=sub, title=anatype+'\n Both scans $\sigma$ = {0:5.3g}'.format(ss), min=-nlo*ss, max=nhi*ss)


def display_all(mapsb, mapsb_pos, mapsb_neg, anatype=''):
    unseen = (mapsb == hp.UNSEEN) | (mapsb_pos == hp.UNSEEN) | (mapsb_neg == hp.UNSEEN)

    ### Average of back and Forth
    mapav = (mapsb_pos + mapsb_neg)/2
    mapav[unseen] = hp.UNSEEN

    ### Difference of back and Forth
    mapdiff = (mapsb_pos - mapsb_neg)
    mapdiff[unseen] = hp.UNSEEN

    ### Difference of All and Av
    mapdiff2 = (mapav - mapsb)
    mapdiff2[unseen] = hp.UNSEEN

    nlo = 3
    nhi = 3
    reso = 12
    mm, ss = ft.meancut(mapsb[~unseen], 3)
    hp.gnomview(mapsb, rot=[0,50], reso=reso, sub=(2,3,1), title=anatype+'\n Both scans $\sigma$ = {0:5.3g}'.format(ss), min=-nlo*ss, max=nhi*ss)
    mmp, ssp = ft.meancut(mapsb_pos[~unseen], 3)
    hp.gnomview(mapsb_pos, rot=[0,50], reso=reso, sub=(2,3,2), title=anatype+'\n Pos scans $\sigma$ = {0:5.3g}'.format(ssp), min=-nlo*ss, max=nhi*ss)
    mmn, ssn = ft.meancut(mapsb_neg[~unseen], 3)
    hp.gnomview(mapsb_neg, rot=[0,50], reso=reso, sub=(2,3,3), title=anatype+'\n Neg scans $\sigma$ = {0:5.3g}'.format(ssn), min=-nlo*ss, max=nhi*ss)
    mma, ssa = ft.meancut(mapav[~unseen], 3)
    hp.gnomview(mapav, rot=[0,50], reso=reso, sub=(2,3,4), title=anatype+'\n Av of Both scans $\sigma$ = {0:5.3g}'.format(ssa), min=-nlo*ss, max=nhi*ss)
    mmd, ssd = ft.meancut(mapdiff[~unseen], 3)
    hp.gnomview(mapdiff, rot=[0,50], reso=reso, sub=(2,3,5), title=anatype+'\n Diff of both scans $\sigma$ = {0:5.3g}'.format(ssd), min=-nlo*ssd, max=nlo*ssd)
    mmd2, ssd2 = ft.meancut(mapdiff2[~unseen], 3)
    hp.gnomview(mapdiff2, rot=[0,50], reso=reso, sub=(2,3,6), title=anatype+'\n Both - Av $\sigma$ = {0:5.3g}'.format(ssd2), min=-nlo*ssd, max=nlo*ssd)


    figure()
    mini = -np.max(mapsb[~unseen])/10
    maxi = np.max(mapsb[~unseen])*0.8

    mm, ss = ft.meancut(mapsb[~unseen], 3)
    hp.gnomview(mapsb, rot=[0,50], reso=reso, sub=(2,3,1), title=anatype+'\n Both scans $\sigma$ = {0:5.3g}'.format(ss), min=mini, max=maxi)
    mmp, ssp = ft.meancut(mapsb_pos[~unseen], 3)
    hp.gnomview(mapsb_pos, rot=[0,50], reso=reso, sub=(2,3,2), title=anatype+'\n Pos scans $\sigma$ = {0:5.3g}'.format(ssp), min=mini, max=maxi)
    mmn, ssn = ft.meancut(mapsb_neg[~unseen], 3)
    hp.gnomview(mapsb_neg, rot=[0,50], reso=reso, sub=(2,3,3), title=anatype+'\n Neg scans $\sigma$ = {0:5.3g}'.format(ssn), min=mini, max=maxi)
    mma, ssa = ft.meancut(mapav[~unseen], 3)
    hp.gnomview(mapav, rot=[0,50], reso=reso, sub=(2,3,4), title=anatype+'\n Av of Both scans $\sigma$ = {0:5.3g}'.format(ssa), min=mini, max=maxi)
    mmd, ssd = ft.meancut(mapdiff[~unseen], 3)
    hp.gnomview(mapdiff, rot=[0,50], reso=reso, sub=(2,3,5), title=anatype+'\n Diff of both scans $\sigma$ = {0:5.3g}'.format(ssd), min=mini, max=maxi)
    mmd2, ssd2 = ft.meancut(mapdiff2[~unseen], 3)
    hp.gnomview(mapdiff2, rot=[0,50], reso=reso, sub=(2,3,6), title=anatype+'\n Both - Av $\sigma$ = {0:5.3g}'.format(ssd2), min=mini, max=maxi)


### Simple map-making: just removing median from TOD

In [ ]:
anatype = 'Raw'
reload(tdt)
###### Pipeline:
# Identify scan types and numbers
scantype_hk, azt, elt, scantype = identify_scans(thk, az, el, tt=tt, doplot=False, thr_speedmin=0.1)

# # remove jumps
# mytod, flags = tdt.jumps_correction(tod)
# Median Scan offset removal
mytod = -tod - np.median(-tod[scantype != 0])

# Map-making
nside = 256
mapsb, mapcount = healpix_map(azt[scantype != 0], elt[scantype != 0], mytod[scantype != 0], nside=nside)
mapsb_pos, _ = healpix_map(azt[scantype > 0], elt[scantype > 0], mytod[scantype > 0], nside=nside)
mapsb_neg, _ = healpix_map(azt[scantype < 0], elt[scantype < 0], mytod[scantype < 0], nside=nside)

# Display Results
display_all(mapsb, mapsb_pos, mapsb_neg, anatype=anatype)

There are obvious issues:
1. significant elevation stripes in the maps: 
     - we need to equalize the average of each back & forth scan offset
2. a clear effect towards the right of the map: likely some "ground pickup" as we see it with the same pattern in the back and forth scans and absent in the subtraction of both.
    - we need to filter the signal in order to remove this or to measure a pattern in azimuth, common to all elevations.
    - It could also be seen with details in thee "no source" scans.
3. Synthesized beeam appears displaced between the back and forth scans:
    - we need to account for time constants
    - a shift in the timestamps has been identified beetween the mount az,el and the data... this is likely the main effect. It is eestimated to be 0.191 seconds (very preliminary)
    
Let's first apply this eempirical timeshift between Mount and TOD (it is a priority to solve this major issue).


In [ ]:
anatype = 'TimeShift corrected'
deltaT= 0.191

###### Pipeline:
# Identify scan types and numbers
scantype_hk, azt, elt, scantype = identify_scans(thk, az, el, tt=tt-deltaT, doplot=False, thr_speedmin=0.1)
# Median Scan offset removal
mytod = -tod - np.median(-tod[scantype != 0])

# Map-making
nside = 256
mapsb, mapcount = healpix_map(azt[scantype != 0], elt[scantype != 0], mytod[scantype != 0], nside=nside)
mapsb_pos, _ = healpix_map(azt[scantype > 0], elt[scantype > 0], mytod[scantype > 0], nside=nside)
mapsb_neg, _ = healpix_map(azt[scantype < 0], elt[scantype < 0], mytod[scantype < 0], nside=nside)

# Display Results
display_all(mapsb, mapsb_pos, mapsb_neg, anatype=anatype)

Most of the difference between two scans has disappeared (not we still DID NOT correct for time-constants).

### Destriping scan by scan
Probably not good enough and will not help for "ground pickup" effect, but very easy to implement. We will just remove a clipped mean or median or the mode from each back and forth scan.

One can play with those options below (and try others), but for now the median leaves some features around the bright peaks and the modee seems unstable. We'll start with the clipped mean.

In [ ]:
def remove_offset_scan(mytod, scantype, method='meancut', apply_to_bad = True):
    ### We remove offsets for each good scan but we also need to remove a coomparable offset for the scantype==0 reggiions in order to keep coninuity 
    ### This si donee by apply_to_bad=True
    
    indices = np.arange(len(mytod))
    last_index = 0
    myoffsetn = 0
    myoffsetp = 0
    donefirst = 0
    
    nscans = np.max(np.abs(scantype))
    for n in range(1, nscans+1):
        # scan +
        ok = scantype == n
        if method == 'meancut':
            myoffsetp, _ = ft.meancut(mytod[ok], 3)
        elif method == 'median':
            myoffsetp = np.median(mytod[ok])
        elif method == 'mode':
            myoffsetp = get_mode(mytod[ok])
        else:
            break
        mytod[ok] -= myoffsetp        
        if apply_to_bad:
            first_index = np.min(indices[ok])
            if (n==1) & (donefirst==0): myoffsetn = myoffsetp ### deal with first region
            vals_offsets = myoffsetn + np.linspace(0,1, first_index-last_index-1)*(myoffsetp-myoffsetn)
            mytod[last_index+1:first_index] -= vals_offsets
            last_index = np.max(indices[ok])
            donefirst = 1
        
        
        # scan -
        ok = scantype == (-n)
        if method == 'meancut':
            myoffsetn, _ = ft.meancut(mytod[ok], 3)
        elif method == 'median':
            myoffsetn = np.median(mytod[ok])
        elif method == 'mode':
            myoffsetn = get_mode(mytod[ok])
        else:
            break
        mytod[ok] -= myoffsetn
        if apply_to_bad:
            first_index = np.min(indices[ok])
            if (n==1) & (donefirst==0): myoffsetp = myoffsetn ### deal with first region
            vals_offsets = myoffsetp + np.linspace(0,1, first_index-last_index-1)*(myoffsetn-myoffsetp)
            mytod[last_index+1:first_index] -= vals_offsets
            last_index = np.max(indices[ok])
            donefirst = 1
    
    return mytod





In [ ]:
anatype = 'TS corr - Az Destriped'

###### Pipelinee:
# 1. Identify scan types and numbers
scantype_hk, azt, elt, scantype = identify_scans(thk, az, el, tt=tt-deltaT, doplot=False, thr_speedmin=0.1)
nscans = np.max(np.abs(scantype))
# 2. Offset removal scan by scan using meancut
mytod = -tod.copy()
mytod = remove_offset_scan(mytod, scantype, method='meancut')

# Map-Making
mapsb, mapcount = healpix_map(azt[scantype != 0], elt[scantype != 0], mytod[scantype != 0], nside=nside)
mapsb_pos, _ = healpix_map(azt[scantype > 0], elt[scantype > 0], mytod[scantype > 0], nside=nside)
mapsb_neg, _ = healpix_map(azt[scantype < 0], elt[scantype < 0], mytod[scantype < 0], nside=nside)

# Display Results
display_all(mapsb, mapsb_pos, mapsb_neg, anatype=anatype)

In [ ]:
# See how the clean TOD looks
plot(tt[scantype>0], mytod[scantype>0], '.', label='Scan +')
plot(tt[scantype<0], mytod[scantype<0], '.', label='Scan -')
plot(tt[scantype==0], mytod[scantype==0], '.', label='Bad')
xlim(0,1000)
ylim(-30000, 30000)
legend()
show()


As expected, it does not affect the ground pickup, but does a good job equalizing scans. We can try to remove a pattern in azimuth, common to all elevations.

(some striping seems still visible and is likely to be due to some 1/f noise - not constant within a scan - that will be reemoved with filtering later on).

### Lets build the "azimuth pattern":
- first we make a profile of the data as a function of azimuth, using the "mode" method. This meeans than in each azimuth bin, the program returns thee mode (maxiimum of thee distribution) of the TOD value for all elevations in this bin. Using the mode is quite powerful for neglecting the contribution from the bright signal peaks.
- then one fits this law with a degree 2 polynomial
- We see below that the profile is slightly different for positive and negative scans, so we do it separately.

In [ ]:
def decorel_azimuth(mytod, azt, scantype, doplot=True):
    ### Profiling in Azimuth
    okall = np.abs(scantype) > 0 
    okpos = scantype > 0 
    okneg = scantype < 0 
    oks = [okpos, okneg]
    oks_names = ['+ scans', '- scans']
    polys = []
    if doplot:
        figure()
    for i in range(len(oks)):
        ok = oks[i]
        minaz = np.min(azt[ok])
        maxaz = np.max(azt[ok])
        xc, yc, dx, dy, _ = ft.profile(azt[ok], mytod[ok], rng=[minaz, maxaz], nbins=25, mode=True, dispersion=True, plot=False)
        z = polyfit(xc, yc, 2, w=1./dy)
        p = np.poly1d(z)
        polys.append(p)
        xaz = np.linspace(minaz, maxaz, 100)
        if doplot:
            pl = errorbar(xc, yc, yerr=dy, xerr=dx, fmt='o')
            plot(xaz, p(xaz), label=oks_names[i], color=pl[0].get_color())
    if doplot:
        xlabel('Azimuth [deg]')
        ylabel('Mode of TOD')
        legend()

    ### Removing the azimuthal effect
    ok = scantype >= 0
    mytod[ok] -= polys[0](azt[ok])
    ok = scantype < 0
    mytod[ok] -= polys[1](azt[ok])
    
    return mytod
    

In [ ]:
anatype = 'TS, Az Corrected'


###### Pipeline:
# 1. Identify scan types and numbers
scantype_hk, azt, elt, scantype = identify_scans(thk, az, el, tt=tt-deltaT, doplot=False, thr_speedmin=0.1)
nscans = np.max(np.abs(scantype))

# 2. Offset removal scan by scan using median 
#    (here we just want to have all scans at the same level before decorrelating from azimuth)
mytod = -tod.copy()
mytod = remove_offset_scan(mytod, scantype, method='median')

# 3. Remove azimuth correlation
mytod = decorel_azimuth(mytod, azt, scantype, doplot=True)

    
# 4. remove offsets again but this time with mode method as it appears to be less affected 
#    by the presence of the peaks (no underestimation of the offset resultingg is shadow around the peaks)
mytod = remove_offset_scan(mytod, scantype, method='mode')


### Then make the maps
mapsb, mapcount = healpix_map(azt[scantype != 0], elt[scantype != 0], mytod[scantype != 0], nside=nside)
mapsb_pos, _ = healpix_map(azt[scantype > 0], elt[scantype > 0], mytod[scantype > 0], nside=nside)
mapsb_neg, _ = healpix_map(azt[scantype < 0], elt[scantype < 0], mytod[scantype < 0], nside=nside)

# Display Results
figure()
display_all(mapsb, mapsb_pos, mapsb_neg, anatype=anatype)

In [ ]:
### Look at cleaned TOD
figure()
plot(tt[scantype>0], mytod[scantype>0], '.', label='Scan +')
plot(tt[scantype<0], mytod[scantype<0], '.', label='Scan +')
plot(tt[scantype==0], mytod[scantype==0], '.', label='Bad')
xlim(0,1000)
ylim(-30000, 30000)
legend()
show()


So one can see significant improvement: RMS of the background reduces from ~1.3e4 to 2.8e3.

However, the simplistic "Azimuth pattern" we fitted now appears to be insufficient. It seems that it also evolves with elevation... so we could measure it by elevation bins and get something better.
So there is still significant room for improvement.

We can also test filtering, but we'll have to avoid signal harmonics, and the "Az/el" bacckground pattern will also be ein those harmonics. Also for this wee'll need to fill the `scantype==0` regions with a constrained realization of noise in order to avoid FFT bouncing. This will be donee in a second time.

### Improving on azimuth/elevation background pattern
There are various ways for investigating this (and what I discuss here is surely not exhaustive):
- Trying to avoid signal with thiis TES and see how this changes with elevation.
- Use a dataset where the calibration source is off and cheeck how the pattern changes from one TES to another (important as it might give us some information on the optical or magnetic origin of this effect).
- With a single dataset we could also use multiple TES and make some median in order to avoid the signal as it will not be present on all TES at the same az/el...

For now, let's try the simplest approach: using a single TES.

In [ ]:
def get_chunks(mytod, scantype, value):
    ### returns chunks corresponding to a given value
    current_chunk = []
    chunk_idx = []
    inchunk = 0
    chunknum = 0
    for i in range(len(scantype)):
        if scantype[i]==value:
            inchunk = 1
            current_chunk.append(i)
        else:
            if inchunk == 1:
                chunknum += 1
                chunk_idx.append([current_chunk[0], current_chunk[len(current_chunk)-1]])
                current_chunk = []
                inchunk = 0
    if inchunk == 1:
        chunk_idx.append([current_chunk[0], current_chunk[len(current_chunk)-1]])
    return chunk_idx


def linear_rescale_chunks(mytod, chunks, sz=1000):
    for i in range(len(chunks)):
        thechunk = chunks[i]
        chunklen = thechunk[1] - thechunk[0]+1
        if thechunk[0] == 0:
            # this is the starting index => just the average
            vals = np.zeros(chunklen) + np.median(mytod[thechunk[1]+1: thechunk[1]+sz]) + np.median(mytod[thechunk[0]:thechunk[1]])
            mytod[thechunk[0]:thechunk[1]+1] -= vals
        elif thechunk[1]==(len(mytod)-1):
            # this is the last one => just the average
            vals = np.zeros(chunklen) + np.median(mytod[thechunk[0]-1-sz: thechunk[0]-1]) + np.median(mytod[thechunk[0]:thechunk[1]])
            mytod[thechunk[0]:thechunk[1]+1] -= vals
        else:
            left = np.median(mytod[thechunk[0]-1-sz: thechunk[0]-1])
            right = np.median(mytod[thechunk[1]+1: thechunk[1]+sz])
            vals = left + np.linspace(0,1, chunklen)*(right-left)
            mytod[thechunk[0]:thechunk[1]+1] -= np.median(mytod[thechunk[0]:thechunk[1]+1]) - vals
            
    return mytod



In [ ]:
def decorel_azel(mytod, azt, elt, scantype, doplot=True, n_el=20, degree=3):
    ### Profiling in Azimuth and elevation
    el_lims = np.linspace(np.min(el)-0.0001, np.max(el)+0.0001, n_el+1)
    el_av = 0.5 * (el_lims[1:] + el_lims[:-1])

    okall = np.abs(scantype) > 0 
    okpos = scantype > 0 
    okneg = scantype < 0 
    oks = [okpos, okneg]
    oks_names = ['+ scans', '- scans']
    minaz = np.min(azt[okall])
    maxaz = np.max(azt[okall])
    xaz = np.linspace(minaz, maxaz, 100)

    if doplot:
        figure()
        xlabel('Azimuth [deg]')
        ylabel('Mode of TOD')
    
    coefficients = np.zeros((2, n_el, degree+1))
    for i in range(len(oks)):
        if doplot: subplot(1,2,i+1)
        for j in range(n_el):
            ok = oks[i] & (elt >= el_lims[j]) & (elt < el_lims[j+1])
            xc, yc, dx, dy, _ = ft.profile(azt[ok], mytod[ok], rng=[minaz, maxaz], nbins=50, mode=True, dispersion=True, plot=False)
            z = polyfit(xc, yc, degree, w=1./dy)
            p = np.poly1d(z)
            coefficients[i,j,:] = z
            if doplot:
                pl = errorbar(xc, yc, yerr=dy, xerr=dx, fmt='o')
                plot(xaz, p(xaz), color=pl[0].get_color(), label = oks_names[i] + ' - El = {0:5.1f}'.format(np.mean(elt[ok])))
    if doplot: legend()

    ### Now interpolate this to remove it to the data
    nscans = np.max(np.abs(scantype))
    for i in range(1, nscans+1):
        okp = scantype == i
        okn = scantype == (-i)
        for ok in [okp, okn]:
            the_el = np.median(elt[ok])
            myp = np.poly1d([np.interp(the_el, el_av, coefficients[0,:,i]) for i in arange(degree+1)])
            mytod[ok] -= myp(azt[ok])
    ### And interpolate for scantype==0 regions
    bad_chunks = get_chunks(mytod, scantype, 0)
    mytod = linear_rescale_chunks(mytod, bad_chunks, sz=100)
    return mytod

In [ ]:
anatype = 'TS, Az/El Corrected'

###### Pipeline:
# 1. Identify scan types and numbers
scantype_hk, azt, elt, scantype = identify_scans(thk, az, el, tt=tt-deltaT, doplot=False, thr_speedmin=0.1)
nscans = np.max(np.abs(scantype))

# 2. Offset removal scan by scan using median 
#    (here we just want to have all scans at the same level before decorrelating from azimuth)
mytod = -tod.copy()
mytod = remove_offset_scan(mytod, scantype, method='median')

# 3. Remove azimuth and elevation correlation
mytod = decorel_azel(mytod, azt, elt, scantype, doplot=True)

    
# 4. remove offsets again but this time with mode method as it appears to be less affected 
#    by the presence of the peaks (no underestimation of the offset resultingg is shadow around the peaks)
mytod = remove_offset_scan(mytod, scantype, method='mode')


### Then make the maps
mapsb, mapcount = healpix_map(azt[scantype != 0], elt[scantype != 0], mytod[scantype != 0], nside=nside)
mapsb_pos, _ = healpix_map(azt[scantype > 0], elt[scantype > 0], mytod[scantype > 0], nside=nside)
mapsb_neg, _ = healpix_map(azt[scantype < 0], elt[scantype < 0], mytod[scantype < 0], nside=nside)

# Display Results
figure()
display_all(mapsb, mapsb_pos, mapsb_neg, anatype=anatype)

In [ ]:
### Look at cleaned TOD
figure()
plot(tt[scantype>0], mytod[scantype>0], '.', label='Scan +')
plot(tt[scantype<0], mytod[scantype<0], '.', label='Scan +')
plot(tt[scantype==0], mytod[scantype==0], '.', label='Bad')
xlim(0,1000)
ylim(-30000, 30000)
legend()
show()


## Filtering
Wee have significantly improved things by decorrelating w.r.t. aziimuth and elevation patterns. However, it is clear that the map is still heavily striped. The source signal is sufficiently strong for these stripes to be unlikely to limit strongly our knowledge of the synthesized beam.  Nevertheless it is a good thing too try to improve further our baselines knowledge.


In a realistic scanning strategy on the sky we would benefit from scanning with many angles for each pixel and this would be a powerful tool for reducingg striping (with optimal mapmaking for instance, but also through other means). Heere by definition wee only in azimuth/elevation  and do not beenefit from sky roation. So we have to deal without this extra-information.

Our pipeline is now:
1. Scans identification
2. Median Scan offset removal
3. Remove Azimuth/Elevation correltion
4. Mode scan offset removal


We will try to improve things using Filtering at the end of the pipeline

The first step here is to identify which are the frequencies (in the TOD domain) that are relevant for the signal we are about to reconstruct. Our signal is the synthesized beam which does not have features smaller that the peak width (around 1 degree FWHM on the sky for the TD, 0.39 degrees for the FI). So this corresponds to features (in terms of sigma) in azimuth of:
$$1 deg  / 2.35 / \cos(50) = 0.66~deg~(el=50)$$

As a consequence we do not want to remove any feature in the TOD larger than this (and this is just a rough approximmation as we span 30-70 degrees).

The angular velocity here is 0.75 degree/sec in azimuth, so this corresponds to 0.88 seconds, or 1.1 Hertz.

This means that we expect no signal beyond 1 Hertz, we can then filter these modes out.

#### However it is better to check this with an actual simulation
We take the theoretical synthesized beam and scan it with out scanning strategy and have a look at the expected TOD for the source alone. We take the theoretical map with a large nside in order to have a good sampling in TOD domain.

In [ ]:
import qubic
from qubicpack.utilities import Qubic_DataDir
# Repository for dictionary and input maps
global_dir = Qubic_DataDir(datafile='instrument.py', datadir='../')
dictfilename = global_dir + '/dicts/pipeline_demo.dict'
d = qubic.qubicdict.qubicDict()
d.read_from_file(dictfilename)
d['config'] = 'TD'
d['nside'] = 1024

s = qubic.QubicScene(d)
q = qubic.QubicMultibandInstrument(d)
sb = q[0].get_synthbeam(s, idet=154)   # this is the QubicSoft number for TES #33 I think...

### and now we rotathe this map to elevation 50
rot_custom = hp.Rotator(rot=[180, 90-50], inv=True)
rot_custom(0,0, lonlat=True)
sbrot = rot_custom.rotate_map_alms(sb)

#sbrot[hp.ud_grade(mapsb, d['nside'])==hp.UNSEEN] = hp.UNSEEN
hp.gnomview(mapsb, rot=[0, 50], reso=12, sub=(2,2,1), title='Measured SB')
hp.gnomview(sbrot, rot=[0, 50], reso=12, sub=(2,2,2), title='Theoretical SB')

#### Now we scan it with the scanning strategy
# index of pixels for each time sample
ips = hp.ang2pix(d['nside'], np.radians(90-elt), np.radians(azt))
tod_th = -sbrot[ips]/np.max(sbrot)

### Put both at the same scale and baseline
tod_th = tod_th * (np.max(tod)-np.min(tod)) + np.median(tod)

subplot(2,1,2)
plot(tt, tod, label='Raw TOD')
plot(tt, tod_th, label='Theoretical TOD')
legend()
xlabel('Time')
ylabel('ADU')
legend()


Now let's look at the power spectrum of the signal alone, and compare to that of the data.

This confirms the above reasonning: we expect no signal beyond 1 Hz, even a bit lower - typically 0.6 Hz.

We also clearly see a few important things:
- there is signal at very low frequency: this si bad news as it means that if we highpass the data, we will loose some signal... So 1/f features are going to be really annoying. We may have to live with them...
- The real data shows a very strong peak at the scanning frequency 0.0063 Hz (calculated below from the data). This IS NOT signal, it is the scan-synchronous spurious signal (correlation with azimuth and elevation) that therefore has to be reduced down to the typical size of the theoretical signal.

In [ ]:
mytod = -tod.copy()
p0, ff = ft.power_spectrum(tt, tod, rebin=True)
pth, ff = ft.power_spectrum(tt, tod_th, rebin=True)

plot(ff, p0, label='Raw')
plot(ff, pth, label='Expected signal')
xscale('log')
yscale('log')
xlabel('Frequency [Hz]')
ylabel('PSD')

# scanning period
scanper = np.min(tt[scantype==60])-np.min(tt[scantype==59])
scanfreq = 1./scanper
axvline(x=scanfreq, ls=':', color='k', label='Scanning Frequency ~{0:5.2g} Hz'.format(scanfreq))

legend()


### Let's try filtering

In [ ]:
anatype = 'TS, Az/El Corrected + Filtering'

###### Pipeline:
# 1. Identify scan types and numbers
scantype_hk, azt, elt, scantype = identify_scans(thk, az, el, tt=tt-deltaT, doplot=False, thr_speedmin=0.1)
nscans = np.max(np.abs(scantype))

# 2. Offset removal scan by scan using median 
#    (here we just want to have all scans at the same level before decorrelating from azimuth)
mytod = -tod.copy()
p0, ff = ft.power_spectrum(tt, mytod, rebin=True)
mytod = remove_offset_scan(mytod, scantype, method='median')
p2, ff = ft.power_spectrum(tt, mytod, rebin=True)

# 3. Remove azimuth and elevation correlation
mytod = decorel_azel(mytod, azt, elt, scantype, doplot=False)
p3, ff = ft.power_spectrum(tt, mytod, rebin=True)

# 4. remove offsets again but this time with mode method as it appears to be less affected 
#    by the presence of the peaks (no underestimation of the offset resultingg is shadow around the peaks)
mytod = remove_offset_scan(mytod, scantype, method='mode')
p4, ff = ft.power_spectrum(tt, mytod, rebin=True)


### Look at cleaned TOD
figure()
plot(tt[scantype>0], mytod[scantype>0], '.', label='Scan +')
plot(tt[scantype<0], mytod[scantype<0], '.', label='Scan +')
plot(tt[scantype==0], mytod[scantype==0], '.', label='Bad')
xlim(0,1000)
ylim(-30000, 30000)
legend()
title('TOD Before')


In [ ]:

def fft_filter(signal_in, sampling, pars, filter_fct, divide=False):
    freqs = np.fft.fftfreq(len(signal_in)) * sampling
    myft = np.fft.fft(signal_in)
    myfilter = filter_fct(freqs, pars)
    if divide:
        myft /= myfilter
    else:
        myft *= myfilter
    signal_out = np.real(np.fft.ifft(myft))
    return signal_out

def brute_force_lopass(ff, flopass):
    myfilter = np.ones(len(ff))
    myfilter[np.abs(ff) > flopass] = 0
    return myfilter

def brute_force_bandpass(ff, fcuts):
    myfilter = np.ones(len(ff))
    myfilter[np.abs(ff) < fcuts[0]] = 0
    myfilter[np.abs(ff) > fcuts[1]] = 0
    return myfilter


In [ ]:
# 5. Low-pass filter the data
sampling = 1./np.median(tt[1:]-tt[:-1])    # Sampling frequency
fmin = sampling/len(tod)
print('fmin = {0:5.2g}'.format(fmin))
fcut = 6e-1 # Hz

mytod_brute_LP = fft_filter(mytod, sampling, fcut, brute_force_lopass)
p5_brute_LP, ff = ft.power_spectrum(tt, mytod_brute_LP, rebin=True)

mytod_butter_LP = ft.butter_bandpass_filter(mytod, sampling/len(tod)/2, fcut, sampling, order=2)
p5_butter_LP, ff = ft.power_spectrum(tt, mytod_butter_LP, rebin=True)

fcuts = [1e-4, fcut]
mytod_brute_BP = fft_filter(mytod, sampling, fcuts, brute_force_bandpass)
p5_brute_BP, ff = ft.power_spectrum(tt, mytod_brute_BP, rebin=True)


figure()
plot(ff, pth, label='Theory')
#plot(ff, p0, label='Raw')
plot(ff, p2, label='After P2 (median scan)')
plot(ff, p3, label='After P3 (Az/El Corr)')
#plot(ff, p4, label='After P4 (Mode Scan)')
plot(ff, p5_brute_LP, label='After P5 (Brute Force Lo-Pass)')
plot(ff, p5_butter_LP, label='After P5 (Butterworth Lo-Pass)')
plot(ff, p5_brute_BP, label='After P5 (Brute Force Band-Pass)')
xscale('log')
yscale('log')
xlabel('Frequency [Hz]')
ylabel('PSD')
legend()


### Let's remark that the azel correlation is very efficient in removing the scan-synchronous peak.

In [ ]:
### Then make the maps
mapsb_brute_LP, mapcount = healpix_map(azt[scantype != 0], elt[scantype != 0], mytod_brute_LP[scantype != 0], nside=nside)
mapsb_butter_LP, mapcount = healpix_map(azt[scantype != 0], elt[scantype != 0], mytod_butter_LP[scantype != 0], nside=nside)
mapsb_brute_BP, mapcount = healpix_map(azt[scantype != 0], elt[scantype != 0], mytod_brute_BP[scantype != 0], nside=nside)

In [ ]:
# Display Results

nlo = 3
nhi = 3

# nlo = 0.5
# nhi = 100

display_one(mapsb, anatype='TS, Az/El Corrected', sub=(2,3,1), nlo=nlo, nhi=nhi)
display_one(mapsb_brute_LP, anatype='Brute LP', sub=(2,3,2), nlo=nlo, nhi=nhi)
display_one(mapsb_butter_LP, anatype='Butter LP', sub=(2,3,3), nlo=nlo, nhi=nhi)
display_one(mapsb_brute_BP, anatype='Brute BP', sub=(2,3,5), nlo=nlo, nhi=nhi)


We see that the gain is not huge. This is due to various effects:
- the presence of the bright peaks. There are ways too overcome this, such as masking the regggions where the signal is strong in the TOD, then repacing them by some constrained noise realization, applying the filtering and putting the signal back. This is usually not too efficient unfortunately.
- The stripes we would like to remove are actually medium-frequency (shorter than a scan but much longer that the HF cutoff we apply) so they are not well filtered by our method... Tiis is where redundancy becomes important. With CMB data, we will have much more crossings with various angles, a much lower signal to noise ratio (therefore no strong peak effect) so it will be much easier to filter. However,  one important thing about filter ing is that you always remove some of the signal with filtering. This has to be accounted for at Cl level by measuring the transfer function of the TOD pipeline and correcting the measured Cl accordingly.

Let's also remark that as anticipated, Band-Pass alters the signal significantly because it has a low-frequency component...
